In [ ]:
import os
import re
import csv
import json
import numpy
import sklearn
import cPickle
import pandas as pd
import numpy as np
import datetime
import shutil
import uuid
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.learning_curve import learning_curve

#Setup environment
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

# deployed_dir
cers_swift_messages_deployed_dir = '/apps/rpauser/portal/services/cers_swift_messages/'

# Class for dumping o/p as json
class cers_swift_sample_messages(object):
  def __init__(self, label, message):
    self.label = label
    self.message = message

  def dump_json(self):
   json_str = json.dumps(self, default=jdefault, indent=4)
   return json_str

# Class for dumping o/p as json
class cers_swift_messages_model_requests(object):
  def __init__(self, uuid, data, prediction, prediction_proba, response):
    self.uuid = str(uuid)
    self.data = data
    self.prediction = prediction
    self.prediction_proba = prediction_proba
    self.response = response

  def dump_json(self, directory):
   f = open(directory + '/' + self.uuid + '.json', 'w')
   json_str = json.dumps(self, default=jdefault, indent=4)
   f.write(json_str)
   f.close()
   return json_str

# Utility for json dumping
def jdefault(o):
  if isinstance(o, set):
    return list(o)
  return o.__dict__

# Utility function to setup directory to persist output
def prep_outdir():
   now = datetime.datetime.now()
   directory =  os.path.join(cers_swift_messages_deployed_dir, 'requests', now.strftime("%Y%m%d"))
   if not os.path.exists(directory):
     os.makedirs(directory)
   return directory

# Split the input to tokens
def split_into_tokens(text):
  text = unicode(text, 'utf8')  # convert bytes into proper unicode
  return TextBlob(text).words

# Convert to lemmas
def split_into_lemmas(text):
  text = unicode(text, 'utf8').lower()
  words = TextBlob(text).words
  return [word.lemma for word in words]

# Remove punctuations, stop words, stemming
def clean_text(text):
  text = re.sub(r"\r", "", text)
  text = re.sub(r"\n", "", text)
  text = text.strip(' ')
  return text

# predict the model
def predict_cers_swift_messages(data):
  prediction_proba = ''
  uuid_ = uuid.uuid4()
  data = clean_text(data)
  arr = [data]
  prediction_ack = cers_swift_messages_model_ack.predict(arr)
  #prediction_ack_proba = cers_swift_messages_model_ack.predict_proba(arr)
  prediction_full = cers_swift_messages_model_full.predict(arr)
  #prediction_full_proba = cers_swift_messages_model_full.predict_proba(arr)
  prediction_partial = cers_swift_messages_model_partial.predict(arr)
  #prediction_partial_proba = cers_swift_messages_model_partial.predict_proba(arr)
  prediction_other = cers_swift_messages_model_other.predict(arr)
  #prediction_other_proba = cers_swift_messages_model_other.predict_proba(arr)

  if prediction_ack[0] == 1:
    prediction = 'Acknowledgment'
    #prediction_proba = str("%0.2f" % (prediction_ack_proba[0][1] *100 )) + '%'
  elif prediction_full[0] == 1:
    prediction = 'Full Details'
    #prediction_proba = str("%0.2f" % (prediction_full_proba[0][1] *100 )) + '%'
  elif prediction_partial[0] == 1:
    prediction = 'Partial Details'
    #prediction_proba = str("%0.2f" % (prediction_partial_proba[0][1] *100 )) + '%'
  else:
    prediction = 'Others'
    #prediction_proba = str("%0.2f" % (prediction_other_proba[0][1] *100 )) + '%'

  req = cers_swift_messages_model_requests(uuid_, data, prediction, prediction_proba, 'processed')
  return req.dump_json(cers_swift_messages_outdir)


# Load saved model
def load_cers_swift_messages_model():
  global cers_swift_messages_outdir
  global cers_swift_messages_model_partial
  global cers_swift_messages_model_full
  global cers_swift_messages_model_ack
  global cers_swift_messages_model_other
  model_loc_partial = cers_swift_messages_deployed_dir + 'output/20180911130543_M02/partial-model.pkl'
  model_loc_full = cers_swift_messages_deployed_dir + 'output/20180911130543_M02/full-model.pkl'
  model_loc_ack = cers_swift_messages_deployed_dir + 'output/20180911130543_M02/ack-model.pkl'
  model_loc_other = cers_swift_messages_deployed_dir + 'output/20180911130543_M02/other-model.pkl'
  cers_swift_messages_outdir = prep_outdir()
  cers_swift_messages_model_partial = cPickle.load(open(model_loc_partial))
  cers_swift_messages_model_full = cPickle.load(open(model_loc_full))
  cers_swift_messages_model_ack = cPickle.load(open(model_loc_ack))
  cers_swift_messages_model_other = cPickle.load(open(model_loc_other))

# Get random records
def get_records():
  df = pd.read_csv(cers_swift_messages_deployed_dir+"input/training.csv", sep=",", encoding='latin-1')
  #df = df.reindex(np.random.permutation(df.index))
  #df = df.head(5)
  random_subset=df.sample(n=5)
  df=random_subset.head()
  return df.to_json(orient='records')


